In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [4]:
import statistics

In [3]:
df1 = df_receipt.groupby('store_cd').amount.var(ddof=1).reset_index()
df1.sort_values(by='amount',ascending=False).head()

,store_cd,amount
28,S13052,441863.252526
31,S14011,306442.242432
42,S14034,297068.392740
5,S13001,295558.842618
12,S13015,295427.197086


P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [10]:
for i in range(0,101,25):
    print(i,'%…',np.percentile(df_receipt['amount'],i))

0 %… 10.0
25 %… 102.0
50 %… 170.0
75 %… 288.0
100 %… 10925.0


P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [16]:
df2 = df_receipt.groupby('store_cd').amount.mean().reset_index()
df2.query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [33]:
df3 = df_receipt.query('~customer_id.str.startswith("Z")',engine='python')
df4 = df3.groupby('customer_id').amount.sum().reset_index()
ave = np.mean(df4['amount'])
print('全顧客の平均値は','{:.2f}'.format(ave))

全顧客の平均値は 2547.74


P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [34]:
df4.head()

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [36]:
df4.query('amount >= @ave').head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


In [39]:
#解2
df4.query('amount >= {}'.format(ave)).head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300
